<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/6_simple-model/hj-simple-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorial/6_simple-model/hj-simple-model.ipynb

# 간단한 모델 만들어서 제출해보기

간단하게 모델 만들어서 제출해볼 예정

## 환경세팅

In [2]:
%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전 라이브러리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
VALIDATION_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed/scenario/validation/"


In [4]:
import math
import os
import uuid
import time

import numpy as np
import pandas as pd
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [5]:
file_paths = tf.io.gfile.glob(VALIDATION_PATH + "*.tfrecord*")

In [6]:
from waymo_open_dataset.protos import scenario_pb2 

# 데이터셋 로드
dataset = tf.data.TFRecordDataset(file_paths, compression_type='')
for data in dataset :
   scenario = scenario_pb2.Scenario()
   scenario.ParseFromString(data.numpy())
   #break

FailedPreconditionError: ignored